# Florence-2: Advancing a Unified Representation for a Variety of Vision Tasks

## Paper

[Florence-2: Advancing a Unified Representation for a Variety of Vision Tasks](https://arxiv.org/abs/2311.06242) es el paper de Florence-2.

## Resumen del paper

Florence-2 es un modelo foundacional de visión con una representación unificada, se basa en prompts, para una variedad de tareas de visión y visión-lenguaje.

Los modelos grandes de visión existentes son buenos en el aprendizaje por transferencia, pero tienen dificultades para realizar una diversidad de tareas con instrucciones simples. Florence-2 fue diseñado para tomar prompts de texto como instrucciones de tareas y generar resultados en forma de texto, detección de objetos, grounding (relacionar palabras o frases de un lenguaje natural con regiones específicas de una imagen) o segmentación

Para poder entrenar el modelo, crearon el dataset FLD-5B, que tiene  5,4 mil millones de anotaciones visuales completas en 126 millones de imágenes.  Este dataset, fue entrenado por dos módulos de procesamiento eficientes.  
El primer módulo utiliza modelos especializados para anotar imágenes de forma colaborativa y autónoma, en vez de el método de anotación única y manual. Múltiples modelos trabajan juntos para llegar a un consenso, que recuerda al concepto de la sabiduría de las multitudes, asegurando una comprensión de la imagen más confiable e imparcial.  
El segundo módulo refina y filtra iterativamente estas anotaciones automatizadas utilizando modelos fundamentales bien entrenados.

El modelo es capaz de realizar una variedad de tareas, como la detección de objetos, el subtitulado y el grounding, todo dentro de un único modelo. La activación de la tarea se logra a través de prompts de texto

Para desarrollar un modelo de base de visión versátil. Para ello el método de entrenamiento del modelo incorpora tres objetivos de aprendizaje distintos, cada uno de los cuales aborda un nivel diferente de granularidad y comprensión semántica:

 * **Las tareas de comprensión a nivel de imagen** capturan la semántica de alto nivel y fomentan una comprensión integral de las imágenes a través de descripciones lingüísticas. Permiten que el modelo comprenda el contexto general de una imagen y capte las relaciones semánticas y los matices contextuales en el dominio del lenguaje. Las tareas ejemplares incluyen la clasificación de imágenes, el subtitulado y la respuesta a preguntas visuales.
 * **Las tareas de reconocimiento a nivel de región/píxel** facilitan la localización detallada de objetos y entidades dentro de las imágenes, capturando las relaciones entre los objetos y su contexto espacial. Las tareas incluyen la detección de objetos, la segmentación y la comprensión de expresiones de referencia.
 * **Las tareas de alineación visual-semántica de grano fino** requieren una comprensión de grano fino tanto del texto como de la imagen. Implica localizar las regiones de la imagen que corresponden a las frases de texto, como objetos, atributos o relaciones. Estas tareas desafían la capacidad de capturar los detalles locales de las entidades visuales y sus contextos semánticos, así como las interacciones entre los elementos textuales y visuales.

Al combinar estos tres objetivos de aprendizaje en un marco de aprendizaje multitarea, el modelo aprende a manejar diferentes niveles de detalle y comprensión semántica.

## Arquitectura

El modelo emplea una arquitectura de secuencia a secuencia (seq2seq), que integra un codificador de imagen y un codificador-decodificador multimodal

![Florence-2 architecture](https://maximofn.com/wp-content/uploads/2024/06/Florence-2_architecture-2.webp)

Como al modelo le van a entrar imágenes y prompts, tiene un codificador de imagen para obtener los embeddings de la imagen, por otro lado se pasan los prompts por un tokenizador y embedding de texto y localización. Se concatenan los embeddings de la imagen y del prompt y se pasa por un trnasformer para obtener los tokens del texto de salida y la localización en la imagen. Finalmente se pasa por un decodificador de texto y localización para obtener los resultados.

Al codificador-decodificador (transformer) del texto más las posiciones le denominan codificador-decodificador multimodal.

Al extender el vocabulario del tokenizador para incluir tokens de ubicación, se permite que el modelo procese información específica de la región del objeto en un formato de aprendizaje unificado, es decir, mediante un único modelo. Esto elimina la necesidad de diseñar cabezas específicas para diferentes tareas y permite un enfoque más centrado en los datos

Crearon 2 modelos, Florence-2 Base y Florence-2 Large. Florence-2 Base tiene 232B de parámetros y Florence-2 Large tiene 771B de parámetros. Cada uno tiene estos tamaños

|Model | Image Encoder (DaViT) | Image Encoder (DaViT) | Image Encoder (DaViT) | Image Encoder (DaViT) | Encoder-Decoder (Transformer) | Encoder-Decoder (Transformer) | Encoder-Decoder (Transformer) | Encoder-Decoder (Transformer) |
|------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- |
|       | dimensions | blocks | heads/groups | #params | encoder layers | decoder layers | dimensions | #params|
|Florence-2-B | [128, 256, 512, 1024] | [1, 1, 9, 1] | [4, 8, 16, 32] | 90M | 6 | 6 | 768 | 140M|
|Florence-2-L | [256, 512, 1024, 2048] | [1, 1, 9, 1] | [8, 16, 32, 64] | 360M | 12 | 12 | 1024 | 410M|

### Codificador de visión

Usaron DaViT como codificador de visión. Procesa una imagen de entrada a embeddings visuales aplanados (N<sub>v</sub>×D<sub>v</sub>), donde N<sub>v</sub> y D<sub>v</sub> representan el número de embeddings y la dimensión de los embeddings visuales, respectivamente.

### Codificador-decodificador multimodal

Utilizaron una arquitectura transformador estándar para procesar los embeddings visuales y de lenguaje

### Objetivo de optimización

Dada una entrada x (combinación de la imagen y el prompt), y el objetivo y, usaron el modelado de lenguaje estándar con pérdida de entropía cruzada para todas las tareas

## Dataset

Crearon el conjunto de datos FLD-5B que incluye 126 millones de imágenes, 500 millones de anotaciones de texto y 1.3 mil millones de anotaciones de texto-región, y 3.6 mil millones de anotaciones de texto-frase-región en diferentes tareas

### Recopilación de las imágenes

Para recopilar las imágenes usaron imágenes de los datasets ImageNet-22k, Object 365, Open Images, Conceptual Captions y LAION

### Etiquetado de las imágenes

El objetivo principal es generar anotaciones que puedan valer para el aprendizaje multitarea de manera efectiva. Para ello crearon tres categorías de anotaciones: texto, pares de texto-región y tripletas de texto-frase-región.

![Florence-2 Image annotations](https://maximofn.com/wp-content/uploads/2024/06/Florence-2_Image_annotations.webp)

El flujo de trabajo de anotación de datos consta de tres fases: (1) anotación inicial empleando modelos especializados, (2) filtrado de datos para corregir errores y eliminar anotaciones irrelevantes, y (3) un proceso iterativo para el refinamiento de datos.

 * **Anotación inicial con modelos especializados**. Usaron etiquetas sintéticas obtenidas de modelos especializados. Estos modelos especializados son una combinación de modelos fuera de línea entrenados en una variedad de conjuntos de datos disponibles públicamente y servicios en línea alojados en plataformas en la nube. Están específicamente diseñados para sobresalir en la anotación de sus respectivos tipos de anotación. Ciertos conjuntos de datos de imágenes contienen anotaciones parciales. Por ejemplo, Object 365 ya incluye cuadros delimitadores anotados por humanos y las categorías correspondientes como anotaciones de texto-región. En esos casos, fusionaron las anotaciones preexistentes con las etiquetas sintéticas generadas por los modelos especializados.

 * **Filtrado y mejora de datos**. Las anotaciones iniciales obtenidas de los modelos especializados, son susceptibles al ruido y la imprecisión. Por lo que implementaron un proceso de filtrado. Se centra principalmente en dos tipos de datos en las anotaciones: datos de texto y de región. En lo que respecta a las anotaciones textuales, desarrollaron una herramienta de análisis basada en SpaCy para extraer objetos, atributos y acciones. Filtraron textos que contienen objetos excesivos, ya que tienden a introducir ruido y pueden no reflejar con precisión el contenido real en las imágenes. Además, evaluaron la complejidad de las acciones y los objetos midiendo su grado de nodo en el árbol de análisis de dependencias. Conservaron textos con una cierta complejidad mínima para garantizar la riqueza de los conceptos visuales en las imágenes. En relación con las anotaciones de región, eliminaron los cuadros ruidosos por debajo de un umbral de puntuación de confianza. También emplearon la supresión no máxima para reducir los cuadros delimitadores redundantes o superpuestos.

 * **Refinamiento iterativo de datos**. Utilizando las anotaciones iniciales filtradas, entrenaron un modelo multitarea que procesa secuencias de datos.

## Entrenameinto

 * Para el entrenamiento usaron [AdamW](https://www.fast.ai/posts/2018-07-02-adam-weight-decay.html) como optimizador, que es una variante de Adam que incluye la regularización L2 en los pesos.
 * Utilizaron un decaimiento en la tasa de aprendizaje del [coseno](https://www.linkedin.com/pulse/learning-rate-cosine-decay-warmup-hold-period-karel-becerra-fppye/). El valor máximo de la tasa de aprendizaje se estableció en 1e-4 y un warmup lineal de 5000 steps.
 * Usaron [Deep-Speed] y precisión mixta para acelerar el entrenamiento.
 * Usaron un tamaño de lote (batch size) de 2048 para Florence-2 Base y 3072 para Florence-2 Large.
 * Hicieron un primer entrenamiento con imágenes de tamaño 184x184 con todas las imágenes del dataset y luego un ajuste de resolución con imágenes de 768x768 con 500 millones de imágenes para el modelo base y 100 millones de imágenes para el modelo large.

## Resultados

### Evaluación zero-shot

Para tareas zero-shot obtuvieron estos resultados

| Method        | #params | COCO Cap. | COCO Cap. | NoCaps | TextCaps | COCO Det. | Flickr30k | Refcoco | Refcoco+ | Refcocog | Refcoco RES |
|-------------- | -------- | :--------: | :--------: | :----: | :------: | :--------: | :-------: | :------: | :-------: | :-------: | :----------: |
|               |          |    test   |    val    |   val  |   val   | val2017 |   test   | test-A  |  test-B   |   val   | test-A  | test-B   |  val   |  test   |  val    |
|               |          |   CIDEr   |   CIDEr   | CIDEr |  CIDEr  |   mAP    |   R@1    | Accuracy | Accuracy | Accuracy |    mIoU     |
| Flamingo [2]  | 80B      |   84.3    |     -      |   -    |    -     |     -      |     -      |    -     |     -      |     -      |      -       |
| Kosmos-2 [60] | 1.6B     |     -      |     -      |   -    |    -     |     -      |  78.7     | 52.3     |  57.4     |  47.3     | 45.5    | 50.7     | 42.2    | 60.6    | 61.7     |    -       |
| Florence-2-B  | 0.23B    |  133.0    |  118.7     |  70.1  |  34.7    |  34.7     |  83.6     | 53.9     |  58.4     |  49.7     | 51.5    | 56.4     | 47.9    | 66.3    | 65.1     |   34.6      |
| Florence-2-L  | 0.77B    |  135.6    |  120.8     |  72.8  |  37.5    |  37.5     |  84.4     | 56.3     |  61.6     |  51.4     | 53.6    | 57.9     | 49.9    | 68.0    | 67.0     |   35.8      |

Como se puede ver Florence-2, tanto el base, como el largue supera a modelos de uno y dos ordenes de magnitud más grandes.

### Modelo generalista con datos supervisados públicos

Ajustaron los modelos Florence-2 añadiendo una colección de conjuntos de datos públicos que cubren tareas a nivel de imagen, región y píxel. Los resultados se pueden ver en las siguientes tablas.

Rendimiento en tareas de subtitulado y VQA:

| Method         | #params | COCO Caption | NoCaps | TextCaps | VQAv2 | TextVQA | VizWiz VQA |
| -------------- | -------- | :-----------: | :----: | :------: | :----: | :------: | :--------: |
|                |          |  Karpathy test |   val  |   val   |  test-dev  | test-dev  |  test-dev   |
|                |          |   CIDEr    | CIDEr |  CIDEr  |   Acc  |   Acc   |    Acc    |
| **Specialist Models** |          |              |        |          |         |          |            |
| CoCa [92]      | 2.1B     |   143.6     | 122.4  |    -     |  82.3   |    -     |     -      |
| BLIP-2 [44]     | 7.8B     |   144.5     | 121.6  |    -     |  82.2   |    -     |     -      |
| GIT2 [78]      | 5.1B     |    145      | 126.9  |  148.6  |  81.7   |  67.3    |   71.0     |
| Flamingo [2]   | 80B     |   138.1     |   -    |    -     |  82.0   |  54.1    |   65.7     |
| PaLI [15]      | 17B     |   149.1     | **127.0**  | 160.0 |  84.3   | 58.8 / 73.1△ | 71.6 / 74.4△ |
| PaLI-X [12]     | 55B     |   **149.2**     | 126.3  | **147 / 163.7** |  **86.0**   | **71.4 / 80.8△** | **70.9 / 74.6△** |
| **Generalist Models** |          |              |        |          |         |          |            |
| Unified-IO [55] | 2.9B     |      -       |  100   |    -     |  77.9   |    -     |   57.4     |
| Florence-2-B   | 0.23B    |   140.0     | 116.7  |  143.9  |  79.7   |  63.6    |   63.6     |
| Florence-2-L   | 0.77B    |   143.3     | 124.9  |  151.1  |  81.7   |  73.5    |   72.6     |

△ Indica que se usó OCR externo como entrada

Rendimiento en tareas a nivel de región y píxel:

| Method         | #params | COCO Det. | Flickr30k | Refcoco | Refcoco+ | Refcocog | Refcoco RES |
| -------------- | -------- | :--------: | :-------: | :------: | :-------: | :-------: | :----------: |
|                |          | val2017 |   test   | test-A  |  test-B   |   val   | test-A  | test-B   |  val   |  test   |  val    |
|                |          |   mAP    |   R@1    | Accuracy | Accuracy | Accuracy |    mIoU     |
| **Specialist Models** |          |         |          |         |          |         |             |
| SeqTR [99]      | -        |    -     |     -      |  83.7    |  86.5     |  81.2     | 71.5    | 76.3     | 64.9    | 74.9    | 74.2     |    -       |
| PolyFormer [49] | -        |    -     |     -      |  90.4    |  92.9     |  87.2     | 85.0    | 89.8     | 78.0    | 85.8    | 85.9     |   76.9      |
| UNINEXT [84]   | 0.74B    |  **60.6**    |     -      |  92.6    |  94.3     |  91.5     | 85.2    | 89.6     | 79.8    | 88.7    | 89.4     |    -       |
| Ferret [90]    | 13B      |    -     |     -      |  89.5    |  92.4     |  84.4     | 82.8    | 88.1     | 75.2    | 85.8    | 86.3     |    -       |
| **Generalist Models** |          |         |          |         |          |         |             |
| UniTAB [88]     | -        |    -     |  **88.6**     |  91.1    |  83.8     |  81.0     | 85.4    | 71.6     | **84.6**    | 84.7    |     -      |    -       |
| Florence-2-B   | 0.23B    |  41.4    |  84.0     |  92.6    |  94.8     |  91.5     | 86.8    | 91.7     | 82.2    | 89.8    | 82.2     |   78.0      |
| Florence-2-L   | 0.77B    |  43.4    |  85.2     |  **93.4**    |  **95.3**     |  **92.0**     | **88.3**    | **92.9**     | 83.6    | **91.2**    | **91.7**     |   **80.5**      |

Resultados de la detección de objetos COCO y segmentación de instancias

| Backbone | Pretrain      | Mask R-CNN  | Mask R-CNN | DINO   |
| :------- | :-----------: | :---------: | :---: | :---: |
|          |               | APb         | APm        | AP |
| ViT-B    | MAE, IN-1k    | 51.6 | 45.9 | 55.0 |
| Swin-B   | Sup IN-1k     | 50.2 |   -   | 53.4 |
| Swin-B   | SimMIM [83]   | 52.3 |   -   |   -   |
| FocalAtt-B| Sup IN-1k     | 49.0 | 43.7 |   -   |
| FocalNet-B| Sup IN-1k     | 49.8 | 44.1 | 54.4 |
| ConvNeXt v1-B | Sup IN-1k     | 50.3 | 44.9 | 52.6 |
| ConvNeXt v2-B | Sup IN-1k     | 51.0 | 45.6 |   -   |
| ConvNeXt v2-B | FCMAE         | 52.9 | 46.6 |   -   |
| DaViT-B   | Florence-2     | **53.6** | **46.4** | **59.2** |

Detección de objetos COCO utilizando Mask R-CNN y DINO

|Pretrain | Frozen stages  | Mask R-CNN | Mask R-CNN | DINO     | UperNet|
|-------- | -------------  | ---------- | -------- | --------| --------|
|         |                | APb | APm | AP | mloU |
|Sup IN1k   | n/a          | 46.7 | 42.0 | 53.7 | 49 |
|UniCL [87] | n/a          | 50.4 | 45.0 | 57.3 | 53.6|
|Florence-2 | n/a          | **53.6** | **46.4** | **59.2** | **54.9**|
|Florence-2 | [1]          | **53.6** | 46.3 | **59.2** | 54.1|
|Florence-2 | [1, 2]       | 53.3 | 46.1 | 59.0 | 54.4|
|Florence-2 | [1, 2, 3]    | 49.5 | 42.9 | 56.7 | 49.6|
|Florence-2 | [1, 2, 3, 4] | 48.3 | 44.5 | 56.1 | 45.9 |

Resultados de segmentación semántica ADE20K

|Backbone | Pretrain | mIoU | ms-mIoU|
|------- | -------- | -------- | --------|
|ViT-B [24] | Sup IN-1k | 47.4 | -|
|ViT-B [24] | MAE IN-1k | 48.1 | -|
|ViT-B [4] | BEiT | 53.6 | 54.1|
|ViT-B [59] | BEiTv2 IN-1k | 53.1 | -|
|ViT-B [59] | BEiTv2 IN-22k | 53.5 | -|
|Swin-B [51] | Sup IN-1k | 48.1 | 49.7|
|Swin-B [51] | Sup IN-22k | - | 51.8|
|Swin-B [51] | SimMIM [83] | - | 52.8|
|FocalAtt-B [86] | Sup IN-1k | 49.0 | 50.5|
|FocalNet-B [85] | Sup IN-1k | 50.5 | 51.4|
|ConvNeXt v1-B [52] | Sup IN-1k | - | 49.9|
|ConvNeXt v2-B [81] | Sup IN-1k | - | 50.5|
|ConvNeXt v2-B [81] | FCMAE | - | 52.1|
|DaViT-B [20] | Florence-2 | **54.9** | **55.5**|

Se puede ver como Florence-2 no es el mejor en algunas de las tareas, aunque en algunas sí, pero está a la altura de los mejores modelos de cada tarea, teniendo uno o dos ordenes de magnitud menos de parámetros que los otros modelos

## Generación de texto